In [ ]:
# pull cdr records from db
import psycopg2

most_recent_db_scrape_datetime = None

cdr_query = "SELECT * FROM cdr_records"
if most_recent_db_scrape_datetime:
    cdr_query += "WHERE created_at >= '%s'" % most_recent_db_scrape_datetime
cdr_query += ";"

connection_params = {
    "dbname": "mobile_network",
    "user": "admin",
    "password": "admin123",
    "host": "localhost",
    "port": 5432
}
conn = psycopg2.connect(**connection_params)
cursor = conn.cursor()

cursor.execute(cdr_query)
data = cursor.fetchall()

col_names_query = """
SELECT column_name
FROM information_schema.columns
WHERE table_name = '%s';
"""
cursor.execute(col_names_query % 'cdr_records')
cdr_record_cols = cursor.fetchall()
cdr_record_cols = [item for sublist in cdr_record_cols for item in sublist]
print(cdr_record_cols)

cursor.close()
conn.close()

In [ ]:
import pandas as pd

df = pd.DataFrame(data, columns=cdr_record_cols)
df = df.drop(['id'], axis=1)
df.info()


In [ ]:
# split into train and validation sets, do not shuffle because it is time-series data

from sklearn.model_selection import train_test_split

cdr_train, cdr_validate = train_test_split(df, train_size=0.8, shuffle=False)

In [4]:
from datetime import datetime

def expand_time_features(my_cdr_df, time_cols):
    def calculate_time_of_day(my_ts, time_column_name):
        time_of_day = [0]*8 # 0-3, 3-6, 6-9, 9-12, 12-15, 15-18, 18-21, 21-24
        # the_hour = datetime.strptime(my_ts, '%Y-%m-%d %H:%M:%S').hour
        the_hour = my_ts.hour
        time_of_day[the_hour // 3] = 1
        time_bucket_cols = [time_column_name+'_time_'+str(i)+'_'+str(i+3) for i in range(0, 24, 3)]
        return pd.Series(time_of_day, index=time_bucket_cols)

    for time_col in time_cols:
        time_buckets_df = my_cdr_df.apply(lambda row: calculate_time_of_day(row[time_col], time_col), axis=1)
        my_cdr_df = my_cdr_df.join(time_buckets_df)

    return my_cdr_df

In [ ]:
# converting timestamps into time-of-day categorical data

cdr_train = expand_time_features(cdr_train, ['timestamp_arrival'])
if not cdr_train['timestamp_departure'].isnull().any():
    cdr_train = expand_time_features(cdr_train, ['timestamp_departure'])
cdr_train = cdr_train.drop(['timestamp_arrival', 'timestamp_departure', 'created_at'], axis=1)
cdr_train.info()

In [ ]:
import numpy as np

numerical_cols = ['user_location_x', 'user_location_y', 'distance', 'speed', 'duration']
for col in numerical_cols:
    cdr_train[col] = pd.to_numeric(cdr_train[col])


In [ ]:
from sklearn.preprocessing import OneHotEncoder

categorical_cols = ['previous_bts_id', 'bts_id', 'imei', 'mcc', 'mnc', 'lac']

onehot_cols = pd.get_dummies(cdr_train[categorical_cols], prefix=categorical_cols).columns

enc = OneHotEncoder(handle_unknown='ignore')
cdr_train_onehot_arr = enc.fit_transform(cdr_train[categorical_cols])
cdr_train_onehot = pd.DataFrame(cdr_train_onehot_arr.todense(), columns=onehot_cols)

cdr_train = cdr_train.drop(categorical_cols, axis=1)
cdr_train = cdr_train.join(cdr_train_onehot)
cdr_train.info()


In [ ]:
cdr_train = cdr_train.dropna(axis=1, how='all')

numerical_cols = np.intersect1d(numerical_cols, cdr_train.select_dtypes(include=[np.number]).columns)
print(numerical_cols)

In [ ]:
import numpy as np
from scipy import stats

NUM_DEVIATIONS = 3

# version a
cdr_train_mean = np.mean(cdr_train[numerical_cols].to_numpy(), axis=0)
cdr_train_std = np.std(cdr_train[numerical_cols].to_numpy(), axis=0)

cdr_train_z_scores = cdr_train[numerical_cols].apply(lambda row: (row - cdr_train_mean) / cdr_train_std, axis=1)

cdr_train_no_outliers = cdr_train[(np.abs(cdr_train_z_scores) < NUM_DEVIATIONS).all(axis=1)]
cdr_train_outliers = cdr_train[(np.abs(cdr_train_z_scores) >= NUM_DEVIATIONS).any(axis=1)]

# version b
outliers = cdr_train[(np.abs(stats.zscore(cdr_train[numerical_cols])) >= NUM_DEVIATIONS).any(axis=1)]

if not cdr_train_outliers.equals(outliers):
    print("something's wrong with the z scores")


In [ ]:
cdr_train_outliers

In [ ]:
#pickleaj 
# cdr_train_mean, cdr_train_std (za racunanje cdr_validate_z_scores)
# OneHotEncoder
# timestamp_ [kad je zscore calculated]
#pickle spremi u analytics/models/

import pickle
from datetime import datetime

data_to_save = (cdr_train_mean, cdr_train_std, enc, onehot_cols, datetime.now())
with open('../models/cdr_analysis.pickle', 'wb') as f:
    pickle.dump(data_to_save, f)

with open('../models/cdr_validate_df.pickle', 'wb') as f:
    pickle.dump(cdr_validate, f)

In [2]:
import pickle

with open('../models/cdr_analysis.pickle', 'rb') as f:
    data = pickle.load(f)

with open('../models/cdr_validate_df.pickle', 'rb') as f:
    cdr_validate = pickle.load(f)

cdr_train_mean, cdr_train_std, enc, onehot_cols, _ = data

In [ ]:
# prvo moram provesti iste tranformacije na cdr_validate kao sto sam napravio na cdr_train
import pandas as pd
import numpy as np

cdr_validate = expand_time_features(cdr_validate, ['timestamp_arrival'])
if not cdr_validate['timestamp_departure'].isnull().any():
    cdr_validate = expand_time_features(cdr_validate, ['timestamp_departure'])
cdr_validate = cdr_validate.drop(['timestamp_arrival', 'timestamp_departure', 'created_at'], axis=1)


numerical_cols = ['user_location_x', 'user_location_y', 'distance', 'speed', 'duration']
for col in numerical_cols:
    cdr_validate[col] = pd.to_numeric(cdr_validate[col])

categorical_cols = ['previous_bts_id', 'bts_id', 'imei', 'mcc', 'mnc', 'lac']

cdr_validate_onehot_arr = enc.transform(cdr_validate[categorical_cols])
cdr_validate_onehot = pd.DataFrame(cdr_validate_onehot_arr.todense(), columns=onehot_cols)

cdr_validate = cdr_validate.drop(categorical_cols, axis=1)
cdr_validate = pd.concat([cdr_validate.reset_index(drop=True), cdr_validate_onehot.reset_index(drop=True)], axis=1)


cdr_validate = cdr_validate.dropna(axis=1, how='all')

numerical_cols = np.intersect1d(numerical_cols, cdr_validate.select_dtypes(include=[np.number]).columns)

cdr_validate_z_scores = cdr_validate[numerical_cols].apply(lambda row: (row - cdr_train_mean) / cdr_train_std, axis=1)

NUM_DEVIATIONS = 3
cdr_validate_no_outliers = cdr_validate[(np.abs(cdr_validate_z_scores) < NUM_DEVIATIONS).all(axis=1)]
cdr_validate_outliers = cdr_validate[(np.abs(cdr_validate_z_scores) >= NUM_DEVIATIONS).any(axis=1)]

######################################################

NameError: name 'expand_time_features' is not defined

In [ ]:
cdr_validate